In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
chat = ChatOpenAI(temperature=0.1)


In [ ]:
# b = chat.predict("How many planets are there?")
# b

template = PromptTemplate.from_template("{country_a}와 {country_b}사이의 거리는 어떻게되나요?")
prompt = template.format(country_a="맥시코", country_b="인도")
chat.predict(prompt)

In [15]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리 전문가입니다. 그리고 당신은 {language}로만 대답을 합니다."),
    ("ai", "안녕하세요 제 이름은 {name}입니다."),
    ("human", "{country_a}와 {country_b}사이의 거리는 어떻게되나요? 그리고 당신의 이름은 무었입니까?")
])
prompt = template.format_messages(language="일본어", name="길동", country_a="일본", country_b="한국")
chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 일본과 한국 사이의 거리는 대략 900km 정도입니다. 제 이름은 길동입니다.')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse(" a ,  b ,  c ,   d")

['a', 'b', 'c', 'd']

In [16]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do Not reply with anything else."),
    ("human", "{question}")
])
# prompt = template.format_messages(
#     max_items=10,
#     question="What are the planets"
# )
prompt = template.format_messages(
    max_items=10,
    question="What are the colors"
)
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)


['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [17]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question":"What are the pokemons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']